# Dependencies

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
%matplotlib inline

# Preprocessing the Data

In [2]:
# read csv into dataframe
filepath = "static/resources/Austin Properties Dataset.csv"
csv_df = pd.read_csv(filepath)

In [3]:
# check the column names
csv_df.columns

Index(['Address', 'Building Name', 'Zip Code', 'Bed', 'Bath', 'Avg SF',
       '# Units', 'Mix %', 'Units Available Units', 'Units Available Percent',
       'Avg Asking Rent/Unit', 'Avg Asking Rent/SF', 'Avg Effective Rent/Unit',
       'Avg Effective Rent/SF', 'Concessions %', 'Year Built',
       'Year Renovated', 'Units', 'Lande (Acres)', 'Vacancy %', 'Washer/Dryer',
       'Walk in Closet', 'Hardwood/Vinyl Floor', 'Walk Score', 'Transit Score',
       '1 Mile Population', '1 Mile Median Household Income',
       'Miles from Domain', 'Miles from Downtown', 'Unnamed: 29'],
      dtype='object')

In [4]:
csv_df = pd.get_dummies(data=csv_df, columns=['Washer/Dryer'])

In [5]:
# select certain variables
original_df = csv_df[['Bed', 'Bath', 'Avg SF', 'Concessions %', 'Year Built', 
   'Walk in Closet', 'Hardwood/Vinyl Floor',
   'Washer/Dryer_Yes but not in unit', 'Washer/Dryer_Yes in unit', 'Washer/Dryer_no',
   'Walk Score', 'Transit Score', '1 Mile Population', '1 Mile Median Household Income',
   'Miles from Domain', 'Miles from Downtown', 
   'Avg Effective Rent/Unit']]

In [6]:
# process na values
austin_properties_df = original_df.dropna()
# check the result of na value processing
austin_properties_df.count()

Bed                                 5468
Bath                                5468
Avg SF                              5468
Concessions %                       5468
Year Built                          5468
Walk in Closet                      5468
Hardwood/Vinyl Floor                5468
Washer/Dryer_Yes but not in unit    5468
Washer/Dryer_Yes in unit            5468
Washer/Dryer_no                     5468
Walk Score                          5468
Transit Score                       5468
1 Mile Population                   5468
1 Mile Median Household Income      5468
Miles from Domain                   5468
Miles from Downtown                 5468
Avg Effective Rent/Unit             5468
dtype: int64

In [7]:
# check the data type
austin_properties_df.dtypes

Bed                                 float64
Bath                                float64
Avg SF                              float64
Concessions %                       float64
Year Built                          float64
Walk in Closet                      float64
Hardwood/Vinyl Floor                float64
Washer/Dryer_Yes but not in unit      uint8
Washer/Dryer_Yes in unit              uint8
Washer/Dryer_no                       uint8
Walk Score                          float64
Transit Score                       float64
1 Mile Population                   float64
1 Mile Median Household Income      float64
Miles from Domain                   float64
Miles from Downtown                 float64
Avg Effective Rent/Unit             float64
dtype: object

In [8]:
# overview of the new dataframe
austin_properties_df.head()

,Bed,Bath,Avg SF,Concessions %,Year Built,Walk in Closet,Hardwood/Vinyl Floor,Washer/Dryer_Yes but not in unit,Washer/Dryer_Yes in unit,Washer/Dryer_no,Walk Score,Transit Score,1 Mile Population,1 Mile Median Household Income,Miles from Domain,Miles from Downtown,Avg Effective Rent/Unit
0,1.0,1.0,560.0,0.01,2016.0,1.0,1.0,0,1,0,88.0,44.0,20115.0,70385.0,12.6,2.1,1528.0
1,1.0,1.0,612.0,0.01,2016.0,1.0,1.0,0,1,0,88.0,44.0,20115.0,70385.0,12.6,2.1,1633.0
2,1.0,1.0,629.0,0.01,2016.0,1.0,1.0,0,1,0,88.0,44.0,20115.0,70385.0,12.6,2.1,1740.0
3,1.0,1.0,774.0,0.01,2016.0,1.0,1.0,0,1,0,88.0,44.0,20115.0,70385.0,12.6,2.1,1778.0
4,1.0,1.0,778.0,0.01,2016.0,1.0,1.0,0,1,0,88.0,44.0,20115.0,70385.0,12.6,2.1,1852.0


In [9]:
austin_properties_df["Year Built"].mean()

1997.693672275055

In [10]:
austin_properties_df["1 Mile Population"].mean()

16014.830651060716

# Determine X and y for Machine Learning

In [11]:
X = austin_properties_df.iloc[:, :-1]
y = austin_properties_df['Avg Effective Rent/Unit']

# Splitting the data into training and testing sets

In [12]:
from sklearn.model_selection import train_test_split

# 80/20 training and testing data split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

In [13]:
# check the shape of training and test data
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(4374, 16)
(1094, 16)
(4374,)
(1094,)


# Model (I) - Linear Regression

In [14]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [15]:
# train the model
lin_model = LinearRegression()
lin_model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [16]:
# model evaluation
print(f"R-quared (R2): {r2_score(y_test, lin_model.predict(X_test))}")

R-quared (R2): 0.5344241825986812


# Model (II) - Polynomial Regression

In [17]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.exceptions import NotFittedError

In [18]:
poly_model = Pipeline([
    ('a', PolynomialFeatures(degree=2)),
    ('b', LinearRegression(fit_intercept=False))
])

In [19]:
poly_model.fit(X_train, y_train)
poly_model.score(X_test, y_test)

0.4487619375607984

# Model (III) Random Forest
## Please note the actual model is: param_grid = {'n_estimators' : list(range(1,999,2))}

In [20]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

forest = RandomForestRegressor()
param_grid = {'n_estimators' : list(range(1,61,2))}
model_rf = GridSearchCV(forest, param_grid, verbose=3)

model_rf.fit(X_train, y_train)
print(model_rf.best_params_)
print(model_rf.best_score_)

y_pred_rf = model_rf.predict(X_test)

C:\Users\yared\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


Fitting 3 folds for each of 30 candidates, totalling 90 fits
[CV] n_estimators=1 ..................................................
[CV] .......... n_estimators=1, score=0.893489599899968, total=   0.0s
[CV] n_estimators=1 ..................................................
[CV] ......... n_estimators=1, score=0.7322722159195127, total=   0.0s
[CV] n_estimators=1 ..................................................
[CV] ......... n_estimators=1, score=0.8185905574950999, total=   0.0s
[CV] n_estimators=3 ..................................................
[CV] ......... n_estimators=3, score=0.9192070329735946, total=   0.0s
[CV] n_estimators=3 ..................................................
[CV] ......... n_estimators=3, score=0.8781117142262292, total=   0.0s
[CV] n_estimators=3 ..................................................
[CV] ......... n_estimators=3, score=0.9122798294985892, total=   0.0s
[CV] n_estimators=5 ..................................................
[CV] ......... n

[CV] ........ n_estimators=39, score=0.9408956587404675, total=   0.6s
[CV] n_estimators=39 .................................................
[CV] ........ n_estimators=39, score=0.9307104103980288, total=   0.6s
[CV] n_estimators=39 .................................................
[CV] ........ n_estimators=39, score=0.9270588836532595, total=   0.7s
[CV] n_estimators=41 .................................................
[CV] ........ n_estimators=41, score=0.9418576026750402, total=   0.6s
[CV] n_estimators=41 .................................................
[CV] ........ n_estimators=41, score=0.9300879816226154, total=   0.9s
[CV] n_estimators=41 .................................................
[CV] ........ n_estimators=41, score=0.9323495556364509, total=   0.8s
[CV] n_estimators=43 .................................................
[CV] ........ n_estimators=43, score=0.9460854087968235, total=   0.8s
[CV] n_estimators=43 .................................................
[CV] .

[Parallel(n_jobs=1)]: Done  90 out of  90 | elapsed:   58.0s finished


{'n_estimators': 55}
0.9388077272551184


In [21]:
# import pickle
rf_model_filepath = '/static/resources/RF999_Model.sav'
#pickle.dump(model_rf, open(rf_model_filepath, 'wb'))

In [22]:
# model_rf = pickle.load(open(rf_model_filepath, 'rb'))

# Model (IV) - Neural Network Model

In [23]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

C:\Users\yared\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [24]:
# define neural network model
model_nn = Sequential()
model_nn.add(Dense(12, input_dim=16, kernel_initializer='random_uniform', activation='relu'))
model_nn.add(Dense(8, kernel_initializer='random_uniform', activation='relu'))
model_nn.add(Dense(4, kernel_initializer='random_uniform', activation='relu'))
model_nn.add(Dense(1, kernel_initializer='random_uniform', activation='relu'))

# compile model
model_nn.compile(loss='mse', optimizer='adam')

In [25]:
model_nn.fit(X_train, y_train, batch_size=300, epochs=10000, 
             verbose=1, 
             callbacks=[EarlyStopping(monitor='loss', min_delta=1000, patience=300)])

Epoch 1/10000
4374/4374 [==============================] - 0s 96us/step - loss: 3782391.8704
Epoch 2/10000
4374/4374 [==============================] - 0s 8us/step - loss: 3782391.6845
Epoch 3/10000
4374/4374 [==============================] - 0s 8us/step - loss: 3782391.7795
Epoch 4/10000
4374/4374 [==============================] - 0s 6us/step - loss: 3782391.7270
Epoch 5/10000
4374/4374 [==============================] - 0s 6us/step - loss: 3782391.6087
Epoch 6/10000
4374/4374 [==============================] - 0s 6us/step - loss: 3782391.6962
Epoch 7/10000
4374/4374 [==============================] - 0s 7us/step - loss: 3782391.5850
Epoch 8/10000
4374/4374 [==============================] - 0s 5us/step - loss: 3782391.6920
Epoch 9/10000
4374/4374 [==============================] - 0s 7us/step - loss: 3782391.7112
Epoch 10/10000
4374/4374 [==============================] - 0s 7us/step - loss: 3782391.7778
Epoch 11/10000
4374/4374 [==============================] - 0s 6us/step - loss

4374/4374 [==============================] - 0s 6us/step - loss: 3782391.7476
Epoch 89/10000
4374/4374 [==============================] - 0s 6us/step - loss: 3782391.6883
Epoch 90/10000
4374/4374 [==============================] - 0s 6us/step - loss: 3782391.7102
Epoch 91/10000
4374/4374 [==============================] - 0s 7us/step - loss: 3782391.8042
Epoch 92/10000
4374/4374 [==============================] - 0s 6us/step - loss: 3782391.8525
Epoch 93/10000
4374/4374 [==============================] - 0s 6us/step - loss: 3782391.7771
Epoch 94/10000
4374/4374 [==============================] - 0s 6us/step - loss: 3782391.7205
Epoch 95/10000
4374/4374 [==============================] - 0s 8us/step - loss: 3782391.7291
Epoch 96/10000
4374/4374 [==============================] - 0s 8us/step - loss: 3782391.5823
Epoch 97/10000
4374/4374 [==============================] - 0s 8us/step - loss: 3782391.6752
Epoch 98/10000
4374/4374 [==============================] - 0s 9us/step - loss: 37823

4374/4374 [==============================] - 0s 7us/step - loss: 3782391.7390
Epoch 176/10000
4374/4374 [==============================] - 0s 7us/step - loss: 3782391.5730
Epoch 177/10000
4374/4374 [==============================] - 0s 6us/step - loss: 3782391.7791
Epoch 178/10000
4374/4374 [==============================] - 0s 6us/step - loss: 3782391.6962
Epoch 179/10000
4374/4374 [==============================] - 0s 5us/step - loss: 3782391.6351
Epoch 180/10000
4374/4374 [==============================] - 0s 6us/step - loss: 3782391.7785
Epoch 181/10000
4374/4374 [==============================] - 0s 7us/step - loss: 3782391.8062
Epoch 182/10000
4374/4374 [==============================] - 0s 6us/step - loss: 3782391.7819
Epoch 183/10000
4374/4374 [==============================] - 0s 6us/step - loss: 3782391.7476
Epoch 184/10000
4374/4374 [==============================] - 0s 7us/step - loss: 3782391.8645
Epoch 185/10000
4374/4374 [==============================] - 0s 7us/step - l

4374/4374 [==============================] - 0s 7us/step - loss: 3782391.7538
Epoch 263/10000
4374/4374 [==============================] - 0s 6us/step - loss: 3782391.7421
Epoch 264/10000
4374/4374 [==============================] - 0s 8us/step - loss: 3782391.7459
Epoch 265/10000
4374/4374 [==============================] - 0s 8us/step - loss: 3782391.7671
Epoch 266/10000
4374/4374 [==============================] - 0s 7us/step - loss: 3782391.6735
Epoch 267/10000
4374/4374 [==============================] - 0s 8us/step - loss: 3782391.6005
Epoch 268/10000
4374/4374 [==============================] - 0s 8us/step - loss: 3782391.7569
Epoch 269/10000
4374/4374 [==============================] - 0s 8us/step - loss: 3782391.7013
Epoch 270/10000
4374/4374 [==============================] - 0s 7us/step - loss: 3782391.7747
Epoch 271/10000
4374/4374 [==============================] - 0s 8us/step - loss: 3782391.7064
Epoch 272/10000
4374/4374 [==============================] - 0s 8us/step - l

In [26]:
# nn_model_filepath = "/Users/apple/Documents/Data Projects/Machine_Learning_Apartment_Rent/static/resources/NN_Model.h5"
# model_nn.save(nn_model_filepath)

# Model (V) - K Nearest Neighbour

In [27]:
from sklearn.neighbors import KNeighborsRegressor
k_neighbour = KNeighborsRegressor()

from sklearn.model_selection import GridSearchCV
param_grid = {'n_neighbors' : list(range(1,999,2))}
model_k_neighbour = GridSearchCV(k_neighbour, param_grid, verbose=3)

model_k_neighbour.fit(X_train, y_train)

model_k_neighbour.best_params_

model_k_neighbour.best_score_

C:\Users\yared\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


Fitting 3 folds for each of 499 candidates, totalling 1497 fits
[CV] n_neighbors=1 ...................................................
[CV] ........... n_neighbors=1, score=0.911738041980998, total=   0.0s
[CV] n_neighbors=1 ...................................................
[CV] .......... n_neighbors=1, score=0.8601791831498506, total=   0.0s
[CV] n_neighbors=1 ...................................................
[CV] .......... n_neighbors=1, score=0.9177267854641082, total=   0.0s
[CV] n_neighbors=3 ...................................................
[CV] .......... n_neighbors=3, score=0.9180071776410968, total=   0.0s
[CV] n_neighbors=3 ...................................................
[CV] ........... n_neighbors=3, score=0.876044709779677, total=   0.0s
[CV] n_neighbors=3 ...................................................
[CV] .......... n_neighbors=3, score=0.9262704478831104, total=   0.0s
[CV] n_neighbors=5 ...................................................
[CV] ........

[CV] ......... n_neighbors=39, score=0.5270143812408595, total=   0.0s
[CV] n_neighbors=41 ..................................................
[CV] ......... n_neighbors=41, score=0.4216079041465304, total=   0.0s
[CV] n_neighbors=41 ..................................................
[CV] ......... n_neighbors=41, score=0.5505857615449867, total=   0.0s
[CV] n_neighbors=41 ..................................................
[CV] ......... n_neighbors=41, score=0.5144654394938694, total=   0.0s
[CV] n_neighbors=43 ..................................................
[CV] ......... n_neighbors=43, score=0.4174737211575657, total=   0.0s
[CV] n_neighbors=43 ..................................................
[CV] .......... n_neighbors=43, score=0.541736157928688, total=   0.0s
[CV] n_neighbors=43 ..................................................
[CV] ......... n_neighbors=43, score=0.5022151279477297, total=   0.0s
[CV] n_neighbors=45 ..................................................
[CV] .

[CV] ........ n_neighbors=79, score=0.33485629781548476, total=   0.0s
[CV] n_neighbors=79 ..................................................
[CV] ......... n_neighbors=79, score=0.4421061176136611, total=   0.0s
[CV] n_neighbors=79 ..................................................
[CV] ......... n_neighbors=79, score=0.4255377373196777, total=   0.0s
[CV] n_neighbors=81 ..................................................
[CV] ........ n_neighbors=81, score=0.33431296915420006, total=   0.0s
[CV] n_neighbors=81 ..................................................
[CV] ......... n_neighbors=81, score=0.4386399780297008, total=   0.0s
[CV] n_neighbors=81 ..................................................
[CV] ......... n_neighbors=81, score=0.4208537616684114, total=   0.0s
[CV] n_neighbors=83 ..................................................
[CV] ......... n_neighbors=83, score=0.3333751106870808, total=   0.0s
[CV] n_neighbors=83 ..................................................
[CV] .

[CV] ....... n_neighbors=117, score=0.42730915409152537, total=   0.0s
[CV] n_neighbors=117 .................................................
[CV] ........ n_neighbors=117, score=0.3688868109108481, total=   0.0s
[CV] n_neighbors=119 .................................................
[CV] ....... n_neighbors=119, score=0.31077923607723157, total=   0.0s
[CV] n_neighbors=119 .................................................
[CV] ........ n_neighbors=119, score=0.4240340113828722, total=   0.0s
[CV] n_neighbors=119 .................................................
[CV] ....... n_neighbors=119, score=0.36765387531028865, total=   0.0s
[CV] n_neighbors=121 .................................................
[CV] ........ n_neighbors=121, score=0.3098896586350194, total=   0.0s
[CV] n_neighbors=121 .................................................
[CV] ....... n_neighbors=121, score=0.42053007683873656, total=   0.0s
[CV] n_neighbors=121 .................................................
[CV] .

...

[CV] ....... n_neighbors=891, score=0.10569170098470361, total=   0.3s
[CV] n_neighbors=891 .................................................
[CV] ....... n_neighbors=891, score=0.17992306792032875, total=   0.3s
[CV] n_neighbors=891 .................................................
[CV] ....... n_neighbors=891, score=0.13505909339884536, total=   0.3s
[CV] n_neighbors=893 .................................................
[CV] ....... n_neighbors=893, score=0.10547298417432403, total=   0.3s
[CV] n_neighbors=893 .................................................
[CV] ....... n_neighbors=893, score=0.17956053199942823, total=   0.4s
[CV] n_neighbors=893 .................................................
[CV] ........ n_neighbors=893, score=0.1348852374922085, total=   0.7s
[CV] n_neighbors=895 .................................................
[CV] ....... n_neighbors=895, score=0.10521409471102139, total=   0.6s
[CV] n_neighbors=895 .................................................
[CV] .

[CV] ....... n_neighbors=929, score=0.17426499220565972, total=   0.4s
[CV] n_neighbors=929 .................................................
[CV] ........ n_neighbors=929, score=0.1318117034063533, total=   0.3s
[CV] n_neighbors=931 .................................................
[CV] ....... n_neighbors=931, score=0.10124629935391671, total=   0.3s
[CV] n_neighbors=931 .................................................
[CV] ....... n_neighbors=931, score=0.17412506067399225, total=   0.3s
[CV] n_neighbors=931 .................................................
[CV] ........ n_neighbors=931, score=0.1316668679153281, total=   0.3s
[CV] n_neighbors=933 .................................................
[CV] ........ n_neighbors=933, score=0.1010621618034695, total=   0.3s
[CV] n_neighbors=933 .................................................
[CV] ....... n_neighbors=933, score=0.17388428771566622, total=   0.4s
[CV] n_neighbors=933 .................................................
[CV] .

[CV] ....... n_neighbors=967, score=0.12873662579303347, total=   0.4s
[CV] n_neighbors=969 .................................................
[CV] ....... n_neighbors=969, score=0.09779961823649364, total=   0.3s
[CV] n_neighbors=969 .................................................
[CV] ....... n_neighbors=969, score=0.16888061851169245, total=   0.3s
[CV] n_neighbors=969 .................................................
[CV] ........ n_neighbors=969, score=0.1286286768272712, total=   0.3s
[CV] n_neighbors=971 .................................................
[CV] ....... n_neighbors=971, score=0.09763594380824571, total=   0.3s
[CV] n_neighbors=971 .................................................
[CV] ....... n_neighbors=971, score=0.16862079653566886, total=   0.3s
[CV] n_neighbors=971 .................................................
[CV] ....... n_neighbors=971, score=0.12851135805266012, total=   0.3s
[CV] n_neighbors=973 .................................................
[CV] .

[Parallel(n_jobs=1)]: Done 1497 out of 1497 | elapsed: 18.2min finished


0.9067741117679614

# Model Comparison

In [28]:
print(f"Linear Model R-quared (R2): {r2_score(y_test, lin_model.predict(X_test))}")
print(f"Polynomial Model R-quared (R2): {poly_model.score(X_test, y_test)}")
print(f"Random Forest Model R-quared (R2): {model_rf.best_score_}")
print(f"Neural Network Model R-quared (R2): {r2_score(y_test, model_nn.predict(X_test))}")
print(f"K Nearest Neighbour Model R-quared (R2): {model_k_neighbour.best_score_}")

Linear Model R-quared (R2): 0.5344241825986812
Polynomial Model R-quared (R2): 0.4487619375607984
Random Forest Model R-quared (R2): 0.9388077272551184
Neural Network Model R-quared (R2): -3.4172826227771242
K Nearest Neighbour Model R-quared (R2): 0.9067741117679614
